In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten

%matplotlib inline
warnings.filterwarnings("ignore")

# Set seeds to make the experiment more reproducible.
# from tensorflow import set_random_seed
from numpy.random import seed
# set_random_seed(1)
seed(1)

Using TensorFlow backend.
/home/edwin/Documents/cross-entropy-services/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/edwin/Documents/cross-entropy-services/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/edwin/Documents/cross-entropy-services/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/edwin/Documents/cross-entropy-services/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519

In [2]:
training_data = pd.read_csv('./test/train.csv', parse_dates=['date'])

In [3]:
training_data.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [4]:
print('Min date from train set: %s' % training_data['date'].min().date())
print('Max date from train set: %s' % training_data['date'].max().date())

Min date from train set: 2013-01-01
Max date from train set: 2017-12-31


In [10]:
lag_size = 90
# print('Max date from train set: %s' % train['date'].max().date())
# print('Max date from test set: %s' % test['date'].max().date())
# print('Forecast lag size', lag_size)

In [ ]:
# daily_sales = training_data.groupby('date', as_index=False)['sales'].sum()
# store_daily_sales = training_data.groupby(['store', 'date'], as_index=False)['sales'].sum()
# item_daily_sales = training_data.groupby(['item', 'date'], as_index=False)['sales'].sum()

In [ ]:
# daily_sales_sc = go.Scatter(x=daily_sales['date'], y=daily_sales['sales'])
# layout = go.Layout(title='Daily sales', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
# fig = go.Figure(data=[daily_sales_sc], layout=layout)
# iplot(fig)

In [ ]:
# store_daily_sales_sc = []
# for store in store_daily_sales['store'].unique():
#     current_store_daily_sales = store_daily_sales[(store_daily_sales['store'] == store)]
#     store_daily_sales_sc.append(go.Scatter(x=current_store_daily_sales['date'], y=current_store_daily_sales['sales'], name=('Store %s' % store)))

# layout = go.Layout(title='Store daily sales', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
# fig = go.Figure(data=store_daily_sales_sc, layout=layout)
# iplot(fig)

In [ ]:
# item_daily_sales_sc = []
# for item in item_daily_sales['item'].unique():
#     current_item_daily_sales = item_daily_sales[(item_daily_sales['item'] == item)]
#     item_daily_sales_sc.append(go.Scatter(x=current_item_daily_sales['date'], y=current_item_daily_sales['sales'], name=('Item %s' % item)))

# layout = go.Layout(title='Item daily sales', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
# fig = go.Figure(data=item_daily_sales_sc, layout=layout)
# iplot(fig)

In [11]:
training_data = training_data[(training_data['date'] >= '2017-01-01')]

In [12]:
train_gp = training_data.sort_values('date').groupby(['item', 'store', 'date'], as_index=False)
train_gp = train_gp.agg({'sales':['mean']})
train_gp.columns = ['item', 'store', 'date', 'sales']
train_gp.head()

,item,store,date,sales
0,1,1,2017-01-01,19
1,1,1,2017-01-02,15
2,1,1,2017-01-03,10
3,1,1,2017-01-04,16
4,1,1,2017-01-05,14


In [13]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [22]:
window = 10
lag = lag_size
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)
series.head()

,item(t-10),store(t-10),sales(t-10),item(t-9),store(t-9),sales(t-9),item(t-8),store(t-8),sales(t-8),item(t-7),...,sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+90),store(t+90),sales(t+90)
10,1.0,1.0,19.0,1.0,1.0,15.0,1.0,1.0,10.0,1.0,...,18.0,1.0,1.0,11.0,1,1,14,1.0,1.0,21.0
11,1.0,1.0,15.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,...,11.0,1.0,1.0,14.0,1,1,17,1.0,1.0,17.0
12,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,...,14.0,1.0,1.0,17.0,1,1,7,1.0,1.0,25.0
13,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,...,17.0,1.0,1.0,7.0,1,1,16,1.0,1.0,15.0
14,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,...,7.0,1.0,1.0,16.0,1,1,29,1.0,1.0,20.0


In [23]:
last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]

In [24]:
columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['item', 'store']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item', 'store']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['item(t)', 'store(t)'], axis=1, inplace=True)

In [25]:
# Label
labels_col = 'sales(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (106446, 11)
Validation set shape (70964, 11)


,sales(t-10),sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
54367,66.0,74.0,81.0,77.0,95.0,104.0,89.0,55.0,74.0,73.0,82
73505,52.0,57.0,97.0,76.0,50.0,63.0,57.0,56.0,56.0,84.0,98
126589,35.0,51.0,59.0,45.0,54.0,71.0,46.0,42.0,55.0,50.0,51
124876,44.0,55.0,58.0,74.0,66.0,72.0,87.0,68.0,62.0,72.0,66
58911,47.0,43.0,55.0,63.0,70.0,43.0,46.0,60.0,44.0,61.0,57


In [30]:
epochs = 1
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [31]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_valid_series.shape)

Train set shape (106446, 11, 1)
Validation set shape (70964, 11, 1)


In [32]:
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(loss='mse', optimizer=adam)
model_cnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 10, 64)            192       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 5, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 320)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                16050     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 16,293
Trainable params: 16,293
Non-trainable params: 0
_________________________________________________________________


In [33]:
cnn_history = model_cnn.fit(X_train_series, Y_train, validation_data=(X_valid_series, Y_valid), epochs=epochs, verbose=2)

Train on 106446 samples, validate on 70964 samples
Epoch 1/1
 - 13s - loss: 437.4221 - val_loss: 415.9752
